## Asteroid conjunction finder

Find close approaches between asteroid positions (obtained dynamically from MPC ephemeris service) and NGC DSO catalog (cached locally from Vizier table)

In [8]:
# Parameters, edit to taste

# Date & time for ephemeris lookup

OBS_TIME = '2025-04-01 20:00:00'

# coords of home location (used to calculate altitude of conjunction)

OBS_LAT = 52.9
OBS_LONG = 0.5

# minimum altitude for conjunctions (anything below is filtered out for visibility)
MIN_VISIBLE_ALT = 10 # degrees

# mimimum angular size of DSO to be visible in image
DSO_MIN_SIZE = 2  # 5% of Seestar S50 FoV width

# maximum magnitude for DSO to be visible 
DSO_MAX_MAG = 15  # approx limiting magnitude for Seestar S50

# Max angular distance between asteroid and DSO
MAX_SEPARATION_ARCMIN = 40 # approx frame width in Seestar S50

# arcsec per pixel for scope (used to convert proper motion to px per hr
ARCSEC_PER_PIXEL = 2.49 # seestar S50

# number of asteroids to check (from list ordered by absolute magnitude)
CHECK_ASTEROIDS = 100

In [2]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
  from google.colab import drive
  drive.mount('/content/drive')
  AST_PATH = '/content/drive/My Drive/Astronomy/asteroids with abs mag below 10.csv'
  NGC_PATH = '/content/drive/My Drive/Astronomy/ngc_catalog.ecsv'
  !pip install astroquery
else:
  AST_PATH = './asteroids with abs mag below 10.csv'
  NGC_PATH = './ngc_catalog.ecsv'

In [3]:
# load list of asteroids to search from file (ordered by absolute magnitude)

import csv

with open(AST_PATH) as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader) # skip header row
    asteroid_names = [row[3] for _, row in zip(range(CHECK_ASTEROIDS), csv_reader)]

In [ ]:
# Fetch ngc catalog from CDS vizier server
# This code only needs to be run once to cache the table locally

# from astroquery.vizier import Vizier

# vizier = Vizier()
# vizier.ROW_LIMIT = -1

# ngc_tables = vizier.get_catalogs('VII/118/ngc2000')

# # save ngc table locally for future reference

# ngc_tables[0].write('./ngc_catalog.ecsv')


In [4]:
# create SkyCoord catalog of ngc DSOs for searching

from astropy.io import ascii

# load locally cached ngc table 
ngc_table = ascii.read(NGC_PATH)

# filter table to size and magnitude limits

filtered_ngc_table = ngc_table[ngc_table['size'] > DSO_MIN_SIZE]
filtered_ngc_table = filtered_ngc_table[filtered_ngc_table['mag'] < DSO_MAX_MAG]

In [5]:
from astropy.coordinates import SkyCoord
import astropy.units as u

ngc_catalog = SkyCoord(ra = filtered_ngc_table['RAB2000'], dec = filtered_ngc_table['DEB2000'],
                       unit = (u.hourangle, u.deg))



In [9]:
# Query MPC Horizons service for asteroid ephemeris data

from astroquery.mpc import MPC
from astropy.table import vstack

ephemerides = [MPC.get_ephemeris(a, start = OBS_TIME, number = 1) for a in asteroid_names]
ephemerides = vstack(ephemerides)

# create SkyCoord catalog for search
ephemerides_cat = SkyCoord(ra = ephemerides['RA'], dec = ephemerides['Dec'], unit = u.deg)

In [ ]:
# Find sky conjunctions of asteroid positions and NGC catalog

from astropy.coordinates import AltAz, EarthLocation

import warnings
warnings.filterwarnings('ignore') # block formatting warning message

home = EarthLocation(lat = OBS_LAT * u.deg, lon = OBS_LONG * u.deg)
home_altaz = AltAz(obstime = OBS_TIME, location = home)

idx_ngc, idx_asteroid, sep2d, _ = ephemerides_cat.search_around_sky(ngc_catalog, seplimit = MAX_SEPARATION_ARCMIN * u.arcmin)

print('Predicted NGC-asteroid conjunctions for ', OBS_TIME, '\n')
for ingc, ia, sep in zip(idx_ngc, idx_asteroid, sep2d):
    ngc = filtered_ngc_table[ingc]
    ngc_loc = ngc_catalog[ingc]
    ephem = ephemerides[ia]
    ephem_coords = ephemerides_cat[ia]
    ngc_mag = filtered_ngc_table[ingc]['mag']
    ast_mag = ephem['V']
    dist = ephem['Delta']
    pm = ephem['Proper motion'] # arcsec per hour
    pm_px = pm/ARCSEC_PER_PIXEL  #px per hour
    altaz_ast =  ephem_coords.transform_to(home_altaz).alt.degree
    altaz_ngc = ngc_loc.transform_to(home_altaz).alt.degree
    skip_count = 0
    name = ngc['Name']
    if name[0] != 'I':
        name = 'NGC ' + name
    if altaz_ast < MIN_VISIBLE_ALT or altaz_ngc < MIN_VISIBLE_ALT:
        skip_count += 1
        continue
    print(f"{name}, {ngc['Type']}, (alt={altaz_ngc:.1f}, mag={ngc_mag:.1f}) <> {asteroid_names[ia]} (alt={altaz_ast:.1f}, mag={ast_mag:.1f}), sep={sep:.2f}, pm={pm:.1f}\"({pm_px:.1f}px)/hr, dist={dist})")
print('\nThere were', skip_count, 'conjunctions below the visibility threshold')
    

Predicted conjunctions for  2025-04-01 20:00:00 

NGC 3185, Gx, (alt=53.7, mag=12.2) <> Gallia (alt=53.8, mag=12.5), sep=0.59 deg, pm=18.0"(7.2px)/hr, dist=2.21)
NGC 3187, Gx, (alt=53.9, mag=13.1) <> Gallia (alt=53.8, mag=12.5), sep=0.48 deg, pm=18.0"(7.2px)/hr, dist=2.21)
NGC 3190, Gx, (alt=53.8, mag=11.0) <> Gallia (alt=53.8, mag=12.5), sep=0.42 deg, pm=18.0"(7.2px)/hr, dist=2.21)
NGC 3189, Gx, (alt=53.8, mag=12.0) <> Gallia (alt=53.8, mag=12.5), sep=0.40 deg, pm=18.0"(7.2px)/hr, dist=2.21)
NGC 3193, Gx, (alt=53.9, mag=10.9) <> Gallia (alt=53.8, mag=12.5), sep=0.34 deg, pm=18.0"(7.2px)/hr, dist=2.21)
NGC 3659, Gx, (alt=42.9, mag=13.0) <> Undina (alt=42.7, mag=12.1), sep=0.13 deg, pm=22.8"(9.2px)/hr, dist=2.604)
NGC 4334, Gx, (alt=26.3, mag=14.0) <> Melpomene (alt=26.7, mag=10.3), sep=0.49 deg, pm=38.2"(15.3px)/hr, dist=1.796)

There were 1 conjunctions below the visibility threshold


In [19]:
# predict conjunctions between asteroids
# self match on ephemerides catalog

from astropy.coordinates import match_coordinates_sky

idx_asteroid, sep2d, _ = match_coordinates_sky(ephemerides_cat, ephemerides_cat, nthneighbor=2)

print('Predicted asteroid-asteroid conjunctions for ', OBS_TIME, '\n')
for ia1, (ia2, sep) in enumerate(zip(idx_asteroid, sep2d)):
    if sep > MAX_SEPARATION_ARCMIN * u.arcmin:
        continue
    ephem_ast1 = ephemerides[ia1]
    ephem_coords_ast1 = ephemerides_cat[ia1]
    ephem_ast2 = ephemerides[ia2]
    ephem_coords_ast2 = ephemerides_cat[ia2]
    mag1 = ephem_ast1['V']
    dist1 = ephem_ast1['Delta']
    mag2 = ephem_ast2['V']
    dist2 = ephem_ast2['Delta']
    pm1 = ephem_ast1['Proper motion'] # arcsec per hour
    pm_px1 = pm1/ARCSEC_PER_PIXEL  #px per hour
    pm2 = ephem_ast2['Proper motion'] # arcsec per hour
    pm_px2 = pm2/ARCSEC_PER_PIXEL  #px per hour
    altaz1 =  ephem_coords_ast1.transform_to(home_altaz).alt.degree
    altaz2 =  ephem_coords_ast2.transform_to(home_altaz).alt.degree
    skip_count = 0
    print(f"{asteroid_names[ia1]} (alt={altaz1:.1f}, mag={mag1:.1f}), pm={pm1:.1f}\"({pm_px1:.1f}px)/hr, dist={dist1}) <> {asteroid_names[ia2]} (alt={altaz2:.1f}, mag={mag2:.1f}), sep={sep:.2f}, pm={pm2:.1f}\"({pm_px2:.1f}px)/hr, dist={dist2})")


Predicted asteroid-asteroid conjunctions for  2025-04-01 20:00:00 

Herculina (alt=-55.8, mag=11.6), pm=46.3"(18.6px)/hr, dist=3.146) <> Euterpe (alt=-54.8, mag=12.6), sep=1.90 deg, pm=50.4"(20.2px)/hr, dist=3.055)
Philomela (alt=-1.1, mag=12.7), pm=54.9"(22.1px)/hr, dist=4.073) <> Eugenia (alt=-2.3, mag=13.5), sep=1.31 deg, pm=57.9"(23.3px)/hr, dist=3.87)
Cybele (alt=53.6, mag=13.2), pm=14.5"(5.8px)/hr, dist=3.433) <> Aquitania (alt=54.8, mag=13.5), sep=1.26 deg, pm=16.1"(6.5px)/hr, dist=3.058)
Euterpe (alt=-54.8, mag=12.6), pm=50.4"(20.2px)/hr, dist=3.055) <> Angelina (alt=-54.0, mag=13.7), sep=1.79 deg, pm=45.3"(18.2px)/hr, dist=3.397)
Hermione (alt=9.0, mag=13.0), pm=25.9"(10.4px)/hr, dist=2.9) <> Nemesis (alt=7.2, mag=12.2), sep=1.89 deg, pm=31.1"(12.5px)/hr, dist=2.121)
Proserpina (alt=49.9, mag=13.0), pm=37.8"(15.2px)/hr, dist=2.834) <> Sirona (alt=50.2, mag=12.8), sep=0.98 deg, pm=46.2"(18.6px)/hr, dist=2.462)
Aquitania (alt=54.8, mag=13.5), pm=16.1"(6.5px)/hr, dist=3.058) <> C